In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine, text
from vertexai.language_models import CodeGenerationModel

In [2]:
df = pd.read_csv('penguins.csv') #Replace with the csv file you want to create a database from or if you have a database, skip

In [3]:
temp_db = create_engine('sqlite:///:memory:', echo = False)

In [4]:
data = df.to_sql(name='Penguins', con=temp_db)

In [5]:
# Get the info of the table using df.info() and pass it as a prefix to the query
def create_prefix(query):
    prefix = f'''
    Return a SQL statement that answers the following query:
{query}


For a table called 'Penguins' with the following properties:
#   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
 
 Example Rows:
(0, 'Adelie', 'Torgersen', 39.1, 18.7, 181.0, 3750.0, 'MALE'),
 (1, 'Adelie', 'Torgersen', 39.5, 17.4, 186.0, 3800.0, 'FEMALE')
 Only return the SQL statement for the query. 
    '''
    return prefix

In [6]:
def user_input():
    query = input('You: ')
    return create_prefix(query)

In [7]:
def clean_sql(sql_statement):
    return sql_statement.replace('```sql','').replace("```",'').replace('\n',' ')

In [8]:
# Add your Google cloud credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "C:\\Users\\nasgh\\Downloads\\llmcredentials.json"
code_gen_model = CodeGenerationModel.from_pretrained('code-bison')

In [9]:
def nlp_assistant():
    print("Bot: Hello! I am your AI Assistant. Ask me anything about the database.")
    print("\n")
    prefix = user_input()
    result = code_gen_model.predict(prefix=prefix)
    sql = clean_sql(result.text)
    with temp_db.connect() as conn:
        print("\nBot: Ok, I am running this SQL Statement:")
        print("\n", sql)
        result = conn.execute(text(sql))
    print('---------')
    print('---------')
    print("Here is the answer to your query")
    print(result.all())

In [10]:
nlp_assistant()

Bot: Hello! I am your AI Assistant. Ask me anything about the database.


You: How many female penguins are there in the database?

Bot: Ok, I am running this SQL Statement:

  SELECT COUNT(*) FROM Penguins WHERE sex = 'FEMALE'; 
---------
---------
Here is the answer to your query
[(165,)]
